In [1]:
import ipywidgets,gzip
import pandas as pd
import geopandas as gpd
from geopandas.plotting import plot_polygon_collection as gpdPlot
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
states = gpd.read_file("./data/cb_2017_us_state_20m.shp")

In [3]:
def baseMap(figsize=(35,18),facecolor=[.9,.9,.9],**kwargs):
    #f,ax = plt.subplots(1,figsize=figsize)
    f = plt.figure(figsize=figsize)
    ax = plt.gca()
    plotShape(states,ax=ax,c=facecolor,**kwargs)
    plt.xlim((-127,-65))
    plt.ylim((25,50))
    plt.xticks([])
    plt.yticks([])
    plt.box("off")
    ax.axis('off')
    return f,ax

def plotShape(df,geofield="geometry",ax=None,c=.8*np.ones(3),colorField=None,cm="viridis",
              vmin=None,vmax=None,pltColorBar=False,
              cbarAxis=[0.1,0.15,0.2,0.025],
              cbarOrientation="horizontal",
              cbarTitle=None,
              sm=None,
              **kwargs):
    #if ax is None:
    #    ax = plt.gca()
    if colorField is None:
        df.plot(ax=ax,color=c,**kwargs)
    else:
        #gpdPlot(ax,df[geofield],facecolor=c,**kwargs)
        df.plot(ax=ax,column=colorField,cmap=cm,vmin=vmin,vmax=vmax,**kwargs)
    if sm is None and not colorField is None:
        if vmin is None: vmin = df[colorField].min()
        if vmax is None: vmax = df[colorField].max()
        norm = plt.Normalize(vmin=vmin, vmax=vmax)
        sm = plt.cm.ScalarMappable(
            cmap=cm,
            norm=norm
        )
        sm.set_array(df[colorField])
        c = sm.to_rgba(df[colorField])
    if pltColorBar:
        cbaxes = plt.gcf().add_axes(cbarAxis) 
        C = plt.colorbar(sm, cax = cbaxes, orientation=cbarOrientation,
                         #                 shrink=.47,pad=.025
        )
        if cbarTitle is None: cbarTitle = colorField
        C.set_label(cbarTitle,fontsize=25)
        C.ax.tick_params(labelsize=25)
        return C
    return None

In [5]:
F = gzip.open("./data/summaryData.geojson.gz")
data = gpd.read_file(F)
F.close()
data = data.set_index(["year","month","2SOC"])
data.head()

st    state  unemployment risk  \
year month 2SOC                                   
2019 1     11    AL  Alabama           0.001429   
           13    AL  Alabama           0.000626   
           15    AL  Alabama           0.000874   
           17    AL  Alabama           0.000417   
           19    AL  Alabama           0.000489   

                                                          geometry  
year month 2SOC                                                     
2019 1     11    POLYGON ((-88.46866 31.89386, -88.46866 31.933...  
           13    POLYGON ((-88.46866 31.89386, -88.46866 31.933...  
           15    POLYGON ((-88.46866 31.89386, -88.46866 31.933...  
           17    POLYGON ((-88.46866 31.89386, -88.46866 31.933...  
           19    POLYGON ((-88.46866 31.89386, -88.46866 31.933...

In [6]:
SOC = {
     'management': '11-0000',
     'business/financial operations': '13-0000',
     'computer/math': '15-0000',
     'architecture/engineering': '17-0000',
     'life/physical/social science': '19-0000',
     'community/social services': '21-0000',
     'legal': '23-0000',
     'education': '25-0000',
     'arts/entertainment': '27-0000',
     'healthcare practitioner': '29-0000',
     'healthcare support': '31-0000',
     'protective services': '33-0000',
     'food prep/serving': '35-0000',
     'building/grounds cleaning/maintenance': '37-0000',
     'personal care': '39-0000',
     'sales/retail': '41-0000',
     'office/admin support': '43-0000',
     'farming/fishing/forestry': '45-0000',
     'construction': '47-0000',
     'installation/maintenance/repair': '49-0000',
     'production': '51-0000',
     'transportation': '53-0000',
     'military specific': '55-0000',
     'INA': 'INA',
     'total': 'total'
}

In [7]:
def plotMap(year,month,soc):
    code = SOC[soc][:2]
    temp = data.loc[year,month,code,:]
    fig,ax = baseMap(edgecolor='k')
    plotShape(temp,colorField="unemployment risk",ax=ax,pltColorBar=True,cm="Purples",cbarTitle="%s\nUnemployment Risk" % soc.capitalize(),
             edgecolor='k')
    plt.title("Month: %d, Year: %d" % (month,year),fontsize=25)

In [8]:
years = data.index.unique("year")
months = data.index.unique("month")
ipywidgets.interact(
    plotMap,
    year=ipywidgets.IntSlider(2019,np.min(years),np.max(years),1),
    month=ipywidgets.IntSlider(4,np.min(months),np.max(months),1),
    soc=ipywidgets.Dropdown(
        options=[f for f in list(SOC.keys()) if not f in ["military specific","total","INA"]],
        value="construction",
        description='Major SOC:',
        disabled=False,
    )
)

interactive(children=(IntSlider(value=2019, description='year', max=2020, min=2019), IntSlider(value=4, descri…

<function __main__.plotMap(year, month, soc)>